## Importing libraries, initialising global variables

In [37]:
!pip install imageio
!pip install pandas
!pip install glob

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [38]:
import imageio
import os
import pandas as pd
from statistics import median
from random import randint
from glob import glob
import pandas as pd
import numpy as np
from keras.layers.core import Flatten, Dropout
from keras.layers import Input, Dense, Lambda, Layer
from keras import backend as K
from keras import applications
from keras.models import Sequential, Model
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet import ResNet152

num_samples = 29000

In [39]:
use_colab=False
wikiHow=True

# Path to folder containing images
if use_colab:
  from google.colab import drive
  
  drive.mount('/content/drive')
  dataset_directory = '/content/drive/MyDrive/Colab/datasets/wikihow'
  workspace = '/content/drive/MyDrive/Colab/wikihow/Siamese'
else:
  dataset_directory = '/kuacc/users/asafa22/google-drive/wikihow'
  workspace = '/kuacc/users/asafa22/workspace/wikihow/Siamese'
  #workspace = "./"

if wikiHow:
    text_image_mapping_file_path = os.path.join(workspace, 'articles.csv')
else:
    text_image_mapping_file_path = os.path.join(dataset_directory, 'keys.csv')

word_vector_file_path = os.path.join(workspace, "word2vec_gensim.csv")


## Generator function

In [40]:

# data generator for neural network
# forms correct and incorrect pairings of images with text descriptions and labels them as correct (1) or incorrect (0)

def generator(batch_size, df):
    
    batch_img = np.zeros((batch_size, 224, 224, 3))
    batch_txt = np.zeros((batch_size, 512))
    batch_labels = np.zeros((batch_size,1))
    
    video_ids = df['image']
    video_txt = df['txt_enc']
    
    length = len(df) -1
    
    while True:
        for i in range(batch_size//2):
            
            i = i*2
            
            #correct
            sample = randint(0,length)
            file = video_ids.iloc[sample]
            
            correct_txt = video_txt.iloc[sample]
            
            im = load_img(file, target_size=(224, 224))
            im = img_to_array(im)
            im = np.expand_dims(im, axis=0)
            im = preprocess_input(im)
            
            batch_img[i-2] = im
            batch_txt[i-2] = correct_txt
            batch_labels[i-2] = 1
                       
            #incorrect 
            file = video_ids.iloc[randint(0,length)]
                       
            im = load_img(file, target_size=(224, 224))
            im = img_to_array(im)
            im = np.expand_dims(im, axis=0)
            im = preprocess_input(im)

            batch_img[i-1] = im
            batch_txt[i-1] = correct_txt
            batch_labels[i-1] = 0
                        
        yield [batch_txt, batch_img], batch_labels

## Utils

In [41]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def create_img_encoder(input_dim, resnet):
    x = Sequential()
    x.add(resnet)
    x.add(Dense(500, activation="relu"))
    x.add(Dropout(0.5))
    x.add(Dense(512, activation="relu"))
    return x

def create_txt_encoder(input_dim):
    x = Sequential()
    x.add(Dense(500, input_shape = (512,), activation="relu"))
    x.add(Dropout(0.5))
    x.add(Dense(512, activation="relu"))
    return x

def compute_accuracy(predictions, labels):
    return labels[predictions.ravel() < 0.5].mean()

## Initialise ResNet152

In [42]:
#resnet = ResNet152(include_top=True, weights='imagenet')
resnet = ResNet152(include_top=True, weights=None)

for layer in resnet.layers:
    layer.trainable = False

## Creating model and loading data

In [43]:
input_txt = Input(shape=(512,))
input_img = Input(shape=(224, 224, 3))

txt_enc = create_txt_encoder(input_txt)
img_enc = create_img_encoder(input_img, resnet)

encoded_txt = txt_enc(input_txt)
encoded_img = img_enc(input_img)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([encoded_txt, encoded_img])

model = Model([input_txt, input_img], distance)

adam = Adam(lr=0.00001)
model.compile(loss=contrastive_loss, optimizer=adam)

model.summary()




Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_15 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential_8 (Sequential)      (None, 512)          513012      ['input_14[0][0]']               
                                                                                                  
 sequential_9 (Sequential)      (None, 512)          61176956    ['input_15[0][0]']         

/kuacc/users/asafa22/.conda/envs/tf/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [44]:
import re
if wikiHow:
    dataset = pd.read_csv(text_image_mapping_file_path, header=0)
    image_pah = dataset['image']
else:
    # The CSV generated by the word2vec(gensim) model
    #img_paths = [DATASET_PATH + str(i) + '.jpg' for i in range(12305)]
    df = pd.read_csv(text_image_mapping_file_path)
    img_paths = df.iloc[:, 1]
    dataset = pd.DataFrame()
    dataset['image'] = pd.Series(img_paths)


# append dataset directory to image paths #
dataset['image'] = dataset_directory +"/"+dataset['image'].astype(str)
# append word vector data to dataset #
data = pd.read_csv(word_vector_file_path, header=None)
data = list(np.array(data))
dataset['txt_enc'] = pd.Series(data)

# shuffle the dataset
dataset = dataset.sample(frac=1).reset_index(drop=True)
# split data into train, validation and test #
df_test = dataset[num_samples:]
dataset = dataset[:num_samples]

df_train = dataset[:int(num_samples*0.8)]
df_val = dataset[int(num_samples*0.8):]

## Training

In [45]:
num_of_epochs = 55
model.fit_generator(generator(30, df_train), steps_per_epoch= int(int(num_samples*0.8)/30), validation_data= generator(30, df_val), validation_steps=int(int(num_samples*0.2)/30), epochs=num_of_epochs, verbose=1)
weights_file = os.path.join(workspace,'weights.h5')
model.save_weights(weights_file)
model.save('model')

/tmp/ipykernel_87641/2745313800.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator(30, df_train), steps_per_epoch= int(int(num_samples*0.8)/30), validation_data= generator(30, df_val), validation_steps=int(int(num_samples*0.2)/30), epochs=num_of_epochs, verbose=1)


Epoch 1/55
 23/773 [..............................] - ETA: 3:00 - loss: 16.9520

2023-01-24 22:19:21.381783: W tensorflow/core/framework/op_kernel.cc:1818] UNKNOWN: FileNotFoundError: [Errno 2] No such file or directory: '/kuacc/users/asafa22/google-drive/wikihow/nan'
Traceback (most recent call last):

  File "/kuacc/users/asafa22/.conda/envs/tf/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/kuacc/users/asafa22/.conda/envs/tf/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/kuacc/users/asafa22/.conda/envs/tf/lib/python3.9/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1039, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/kuacc/users/asafa22/.conda/envs/tf/lib/python3.9/site-packages/keras/engine/data_adapter.py", line 901, in wrapped_generator
    for data in generator_fn():

  File "/tmp/ipykernel_87641/4150225356.py", line 26, in generator
    im =

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  FileNotFoundError: [Errno 2] No such file or directory: '/kuacc/users/asafa22/google-drive/wikihow/nan'
Traceback (most recent call last):

  File "/kuacc/users/asafa22/.conda/envs/tf/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/kuacc/users/asafa22/.conda/envs/tf/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/kuacc/users/asafa22/.conda/envs/tf/lib/python3.9/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1039, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/kuacc/users/asafa22/.conda/envs/tf/lib/python3.9/site-packages/keras/engine/data_adapter.py", line 901, in wrapped_generator
    for data in generator_fn():

  File "/tmp/ipykernel_87641/4150225356.py", line 26, in generator
    im = load_i

## Load saved weights

In [ ]:
# Load from where you stored the weights
model.load_weights(weights_file)

## Decide size of test set

In [ ]:
subset_size = 200
subset = df_test.iloc[:subset_size]

## Metrics

In [ ]:
# metrics - img -> text

mr = []
top_1_count = 0
top_5_count = 0
top_10_count = 0

for i in range(subset_size):
    file = subset['image'].iloc[i]
    im = load_img(file, target_size=(224, 224))
    im = img_to_array(im)
    im = np.expand_dims(im, axis=0)
    im = preprocess_input(im)
    
    image_array = np.zeros((subset_size, 224, 224, 3))
    for k in range(subset_size):
        image_array[k] = im
        
    txt_array = np.zeros((subset_size, 512))
    for j in range(subset_size):
        txt = subset['txt_enc'].iloc[j]
        txt_array[j] = txt
    
    predictions = [pred[0] for pred in model.predict([txt_array, image_array])]
    pred_i = predictions[i]
    predictions.sort()
    rank = predictions.index(pred_i)
    if rank < 10:
        top_10_count += 1
    if rank < 5:
        top_5_count += 1
    if rank < 1:
        top_1_count += 1
    mr.append(rank+1)     

print('Median Rank(img->txt):', median(mr)*100/subset_size, '%')
print('R@1(img->txt):', top_1_count*100/subset_size, '%')
print('R@5(img->txt):', top_5_count*100/subset_size, '%')
print('R@10(img->txt):', top_10_count*100/subset_size, '%')

In [ ]:
# metrics - txt -> img

mr = []
top_1_count = 0
top_5_count = 0
top_10_count = 0

for i in range(subset_size):
    txt = subset['txt_enc'].iloc[i] 
    txt_array = np.zeros((subset_size, 512))
    for k in range(subset_size):
        txt_array[k] = txt
        
        
    image_array = np.zeros((subset_size, 224, 224, 3))
    for j in range(subset_size):
        file = subset['image'].iloc[j]
        im = load_img(file, target_size=(224, 224))
        im = img_to_array(im)
        im = np.expand_dims(im, axis=0)
        im = preprocess_input(im)
        image_array[k] = im
    
    predictions = [pred[0] for pred in model.predict([txt_array, image_array])]
    pred_i = predictions[i]
    predictions.sort()
    rank = predictions.index(pred_i)
    if rank < 10:
        top_10_count += 1
    if rank < 5:
        top_5_count += 1
    if rank < 1:
        top_1_count += 1
    mr.append(rank+1)     

print('Median Rank(txt->img):', median(mr)*100/subset_size, '%')
print('R@1(txt->img):', top_1_count*100/subset_size, '%')
print('R@5(txt->img):', top_5_count*100/subset_size, '%')
print('R@10(txt->img):', top_10_count*100/subset_size, '%')

## Download Weights

In [ ]:
# download weights

from IPython.display import FileLink

FileLink(weights_file)

## Try predicting

In [ ]:
# trying out predict

text = np.zeros((2, 512))
image = np.zeros((2, 224, 224, 3))

file = dataset['image'].iloc[21]     
correct_txt = dataset['txt_enc'].iloc[21]

im = load_img(file, target_size=(224, 224))
im = img_to_array(im)
im = np.expand_dims(im, axis=0)
im = preprocess_input(im)

image[0] = im

text[0] = correct_txt

file = dataset['image'].iloc[21]     
correct_txt = dataset['txt_enc'].iloc[90]

im = load_img(file, target_size=(224, 224))
im = img_to_array(im)
im = np.expand_dims(im, axis=0)
im = preprocess_input(im)

image[1] = im

text[1] = correct_txt

model.predict([text, image])